# Import Library

In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import sys
import joblib

# Fungsi Pembantu: Mengelompokkan Kategori Langka

In [12]:
def group_rare_categories(df, column_name, top_n=50, other_category_name=None):
    """
    Mengelompokkan kategori yang jarang muncul dalam sebuah kolom
    menjadi satu kategori 'Other_column_name'.

    Args:
        df (pd.DataFrame): DataFrame input.
        column_name (str): Nama kolom kategorikal yang akan diproses.
        top_n (int): Jumlah kategori teratas yang akan dipertahankan.
                     Semua kategori lain akan digabungkan.
        other_category_name (str, optional): Nama kategori untuk nilai yang digabungkan.
                                             Defaultnya adalah f'Other_{column_name}'.
    Returns:
        pd.DataFrame: DataFrame dengan kolom yang sudah dikelompokkan.
    """
    if column_name not in df.columns:
        print(f"Peringatan: Kolom '{column_name}' tidak ditemukan untuk pengelompokan kategori langka.")
        return df

    if other_category_name is None:
        other_category_name = f'Other_{column_name}'

    # Hitung frekuensi setiap kategori
    value_counts = df[column_name].value_counts()
    
    # Identifikasi kategori teratas
    top_categories = value_counts.nlargest(top_n).index
    
    # Ganti kategori yang bukan top_n dengan 'Other_{column_name}'
    df[column_name] = df[column_name].apply(lambda x: x if x in top_categories else other_category_name)
    
    print(f"  - Kolom '{column_name}': Kardinalitas dikurangi menjadi {df[column_name].nunique()} setelah mengelompokkan kategori langka (Top {top_n}).")
    return df

# Definisikan Jalur File Input dan Output

In [13]:
input_csv_base = '../data/base_processed_data.csv'
output_csv_supervised = '../data/supervised_ready_data.csv'
preprocessor_output_path = '../models/preprocessor_supervised.pkl'

print(f"Jalur Input CSV: {input_csv_base}")
print(f"Jalur Output CSV: {output_csv_supervised}")
print(f"Jalur Output Preprocessor: {preprocessor_output_path}")

Jalur Input CSV: ../data/base_processed_data.csv
Jalur Output CSV: ../data/supervised_ready_data.csv
Jalur Output Preprocessor: ../models/preprocessor_supervised.pkl


# Muat Data Dasar dan Analisis Kardinalitas Awal

In [14]:
print(f"--- MEMULAI PROSES PREPROCESSING SUPERVISED ---")
print(f"Python Executable: {sys.executable}")
print(f"Versi Pandas: {pd.__version__}")
print(f"Versi NumPy: {np.__version__}")

print(f"\nMemuat data dasar dari: {input_csv_base}")
df = pd.read_csv(input_csv_base)
print(f"Data dasar dimuat. Ukuran: {df.shape}")

print("\nKardinalitas kolom kategorikal sebelum pengelompokan:")
for col in df.select_dtypes(include='object').columns:
    print(f"Kolom '{col}': {df[col].nunique()} nilai unik.")

--- MEMULAI PROSES PREPROCESSING SUPERVISED ---
Python Executable: c:\Users\alyar\AppData\Local\Programs\Python\Python313\python.exe
Versi Pandas: 2.2.3
Versi NumPy: 2.2.6

Memuat data dasar dari: ../data/base_processed_data.csv
Data dasar dimuat. Ukuran: (558837, 14)

Kardinalitas kolom kategorikal sebelum pengelompokan:
Kolom 'make': 96 nilai unik.
Kolom 'model': 973 nilai unik.
Kolom 'trim': 1963 nilai unik.
Kolom 'body': 87 nilai unik.
Kolom 'transmission': 4 nilai unik.
Kolom 'state': 64 nilai unik.
Kolom 'color': 46 nilai unik.
Kolom 'interior': 17 nilai unik.
Kolom 'seller': 14263 nilai unik.


# Pengelompokan Kategori Langka

In [15]:
print("\nMelakukan pengelompokan kategori langka untuk mengurangi dimensi...")

# Terapkan pengelompokan untuk kolom-kolom kardinalitas tinggi
df = group_rare_categories(df, 'seller', top_n=25) # Dari 973 menjadi ~100+1
df = group_rare_categories(df, 'model', top_n=25)  # Dari 973 menjadi ~100+1
df = group_rare_categories(df, 'trim', top_n=25)   # Dari 973 menjadi ~100+1

# Pertimbangkan juga untuk kolom lain jika 
# nilai uniknya masih terlalu tinggi setelahnya untuk OHE
df = group_rare_categories(df, 'make', top_n=20) # Dari 51 menjadi ~20+1
df = group_rare_categories(df, 'state', top_n=15) # Dari 51 menjadi ~20+1
df = group_rare_categories(df, 'color', top_n=10) # Dari 16 menjadi ~10+1
df = group_rare_categories(df, 'body', top_n=5)  # Dari 16 menjadi ~10+1

print("\nKardinalitas kolom kategorikal setelah pengelompokan:")
for col in df.select_dtypes(include='object').columns:
    print(f"Kolom '{col}': {df[col].nunique()} nilai unik.")


Melakukan pengelompokan kategori langka untuk mengurangi dimensi...
  - Kolom 'seller': Kardinalitas dikurangi menjadi 26 setelah mengelompokkan kategori langka (Top 25).
  - Kolom 'model': Kardinalitas dikurangi menjadi 26 setelah mengelompokkan kategori langka (Top 25).
  - Kolom 'trim': Kardinalitas dikurangi menjadi 26 setelah mengelompokkan kategori langka (Top 25).
  - Kolom 'make': Kardinalitas dikurangi menjadi 21 setelah mengelompokkan kategori langka (Top 20).
  - Kolom 'state': Kardinalitas dikurangi menjadi 16 setelah mengelompokkan kategori langka (Top 15).
  - Kolom 'color': Kardinalitas dikurangi menjadi 11 setelah mengelompokkan kategori langka (Top 10).
  - Kolom 'body': Kardinalitas dikurangi menjadi 6 setelah mengelompokkan kategori langka (Top 5).

Kardinalitas kolom kategorikal setelah pengelompokan:
Kolom 'make': 21 nilai unik.
Kolom 'model': 26 nilai unik.
Kolom 'trim': 26 nilai unik.
Kolom 'body': 6 nilai unik.
Kolom 'transmission': 4 nilai unik.
Kolom 'state':

# Identifikasi Kolom, Label Encoding, dan Pemisahan Fitur/Target

In [ ]:
# --- Identifikasi Kolom (Setelah Pengelompokan Kategori Langka) ---
initial_numerical_cols = df.select_dtypes(include=np.number).columns.tolist()
initial_categorical_cols = df.select_dtypes(include='object').columns.tolist()

# Kolom yang awalnya kardinalitas tinggi, kini sudah dikelompokkan
high_cardinality_cols = ['model', 'trim', 'seller']
low_cardinality_cols = [col for col in initial_categorical_cols if col not in high_cardinality_cols]

print(f"\nKolom Numerik (awal): {initial_numerical_cols}")
print(f"Kolom Kategorikal Kardinalitas Rendah/Sedang (untuk OHE): {low_cardinality_cols}")
print(f"Kolom Kategorikal Kardinalitas Tinggi (untuk Label Encoding): {high_cardinality_cols}")

# --- Label Encoding untuk Kolom Kardinalitas Tinggi ---
print("\nMelakukan Label Encoding untuk kolom kardinalitas tinggi (setelah pengelompokan)...")

# Simpan LabelEncoders dalam sebuah dictionary untuk penggunaan kembali
label_encoders: dict[str, LabelEncoder] = {}
for col in high_cardinality_cols:
    if col in df.columns:
        le = LabelEncoder()
        # Penting: Isi NaN sebelum encoding agar tidak error dan menjadi kategori sendiri
        df[col] = le.fit_transform(df[col].astype(str).fillna('NaN_Category'))
        label_encoders[col] = le 
        print(f"  - Kolom '{col}' di-Label Encode.")
    else:
        print(f"  - Peringatan: Kolom '{col}' tidak ditemukan di DataFrame.")

# --- Pisahkan Fitur (X) dan Target (y) ---
X = df.drop(columns=['sellingprice'])
y = df['sellingprice']

print(f"\nDEBUG_INFO: Shape of X (fitur) sebelum transformasi preprocessor: {X.shape}")
print(f"DEBUG_INFO: Kolom di X: {X.columns.tolist()}")
print(f"DEBUG_INFO: Dtypes of X: \n{X.dtypes}")


Kolom Numerik (awal): ['year', 'condition', 'odometer', 'mmr', 'sellingprice']
Kolom Kategorikal Kardinalitas Rendah/Sedang (untuk OHE): ['make', 'body', 'transmission', 'state', 'color', 'interior']
Kolom Kategorikal Kardinalitas Tinggi (untuk Label Encoding): ['model', 'trim', 'seller']

Melakukan Label Encoding untuk kolom kardinalitas tinggi (setelah pengelompokan)...
  - Kolom 'model' di-Label Encode.
  - Kolom 'trim' di-Label Encode.
  - Kolom 'seller' di-Label Encode.

DEBUG_INFO: Shape of X (fitur) sebelum transformasi preprocessor: (558837, 13)
DEBUG_INFO: Kolom di X: ['year', 'make', 'model', 'trim', 'body', 'transmission', 'state', 'condition', 'odometer', 'color', 'interior', 'seller', 'mmr']
DEBUG_INFO: Dtypes of X: 
year              int64
make             object
model             int64
trim              int64
body             object
transmission     object
state            object
condition       float64
odometer        float64
color            object
interior         ob

# Buat dan Terapkan ColumnTransformer

In [17]:
# --- Perbarui Daftar Kolom Numerik untuk Scaling ---
# Kolom numerik asli + kolom yang baru di-Label Encode
all_numerical_cols_for_scaler = [col for col in initial_numerical_cols if col != 'sellingprice'] + high_cardinality_cols

print(f"\nDEBUG_INFO: Kolom Numerik untuk StandardScaler ({len(all_numerical_cols_for_scaler)}): {all_numerical_cols_for_scaler}")
print(f"DEBUG_INFO: Kolom Kategorikal untuk OneHotEncoder ({len(low_cardinality_cols)}): {low_cardinality_cols}")

# --- Buat Preprocessing Pipeline dengan ColumnTransformer ---
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first')) 
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, all_numerical_cols_for_scaler),
        ('cat', categorical_transformer, low_cardinality_cols)
    ],
    remainder='passthrough'
)

# --- Terapkan Transformasi Fitur ---
print("\nMenerapkan transformasi fitur dengan ColumnTransformer...")
X_processed_array = preprocessor.fit_transform(X)

print(f"DEBUG_INFO: Shape of X_processed_array setelah preprocessor: {X_processed_array.shape}")


DEBUG_INFO: Kolom Numerik untuk StandardScaler (7): ['year', 'condition', 'odometer', 'mmr', 'model', 'trim', 'seller']
DEBUG_INFO: Kolom Kategorikal untuk OneHotEncoder (6): ['make', 'body', 'transmission', 'state', 'color', 'interior']

Menerapkan transformasi fitur dengan ColumnTransformer...
DEBUG_INFO: Shape of X_processed_array setelah preprocessor: (558837, 76)


# Konversi ke Dense Array dan Dapatkan Nama Kolom

In [18]:
# --- Konversi Sparse Matrix ke Dense Array ---
print("\nMengonversi sparse matrix ke dense array (jika diperlukan)...")
if hasattr(X_processed_array, "toarray"):
    X_processed_dense = X_processed_array.toarray()
else:
    X_processed_dense = X_processed_array
print(f"DEBUG_INFO: Shape of X_processed_dense setelah konversi: {X_processed_dense.shape}")

# --- Dapatkan Nama Kolom Setelah Preprocessing ---
scaled_feature_names = all_numerical_cols_for_scaler
ohe_feature_names = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(low_cardinality_cols)

all_transformed_by_name = set(all_numerical_cols_for_scaler + low_cardinality_cols)
remainder_feature_names = [col for col in X.columns if col not in all_transformed_by_name]

processed_feature_names = list(scaled_feature_names) + list(ohe_feature_names) + list(remainder_feature_names)

print(f"\nDEBUG_INFO: Jumlah scaled_feature_names: {len(scaled_feature_names)}")
print(f"DEBUG_INFO: Jumlah ohe_feature_names: {len(ohe_feature_names)}")
print(f"DEBUG_INFO: Jumlah remainder_feature_names: {len(remainder_feature_names)}")
print(f"DEBUG_INFO: Total processed_feature_names: {len(processed_feature_names)}")

# --- Validasi Akhir sebelum membuat DataFrame ---
if X_processed_dense.shape[1] != len(processed_feature_names):
    print(f"\n!!! PERINGATAN: Jumlah kolom tidak sesuai !!!")
    raise ValueError("Jumlah kolom dalam array yang diproses tidak sesuai dengan jumlah nama kolom yang diberikan.")


Mengonversi sparse matrix ke dense array (jika diperlukan)...
DEBUG_INFO: Shape of X_processed_dense setelah konversi: (558837, 76)

DEBUG_INFO: Jumlah scaled_feature_names: 7
DEBUG_INFO: Jumlah ohe_feature_names: 69
DEBUG_INFO: Jumlah remainder_feature_names: 0
DEBUG_INFO: Total processed_feature_names: 76


# Buat DataFrame, Optimasi Tipe Data, dan Simpan Hasil

In [ ]:
# Buat DataFrame dari array fitur yang telah diproses
df_supervised_processed = pd.DataFrame(X_processed_dense, columns=processed_feature_names)

# Tambahkan kembali kolom target
df_supervised_processed['sellingprice'] = y.reset_index(drop=True)

# --- OPTIMASI TIPE DATA UNTUK MENGURANGI UKURAN FILE ---
print("\nMelakukan optimasi tipe data untuk mengurangi ukuran memori dan file...")
initial_memory_usage = df_supervised_processed.memory_usage(deep=True).sum() / (1024**2) # MB
print(f"Ukuran memori awal DataFrame: {initial_memory_usage:.2f} MB")

for col in df_supervised_processed.columns:
    if df_supervised_processed[col].dtype == 'float64':
        df_supervised_processed[col] = df_supervised_processed[col].astype('float32')
    elif df_supervised_processed[col].dtype == 'int64':
        # Perhatikan: sellingprice bisa jadi besar, jadi jangan dipaksa ke int8/int16 jika tidak muat
        if col != 'sellingprice': 
            min_val = df_supervised_processed[col].min()
            max_val = df_supervised_processed[col].max()
            if min_val >= np.iinfo(np.int8).min and max_val <= np.iinfo(np.int8).max:
                df_supervised_processed[col] = df_supervised_processed[col].astype('int8')
            elif min_val >= np.iinfo(np.int16).min and max_val <= np.iinfo(np.int16).max:
                df_supervised_processed[col] = df_supervised_processed[col].astype('int16')
            elif min_val >= np.iinfo(np.int32).min and max_val <= np.iinfo(np.int32).max:
                df_supervised_processed[col] = df_supervised_processed[col].astype('int32')

final_memory_usage = df_supervised_processed.memory_usage(deep=True).sum() / (1024**2)
print(f"Ukuran memori akhir DataFrame: {final_memory_usage:.2f} MB")
print(f"Penghematan memori: {((initial_memory_usage - final_memory_usage) / initial_memory_usage) * 100:.2f}%")

print(f"Menyimpan data yang telah diproses ke: {output_csv_supervised} (CSV)")
df_supervised_processed.to_csv(output_csv_supervised, index=False)
print(f"Ukuran data yang disimpan (CSV): {df_supervised_processed.shape}")

# Tampilkan beberapa baris pertama untuk konfirmasi
print("\nContoh data yang telah diproses (5 baris pertama):")
df_supervised_processed.head()


Melakukan optimasi tipe data untuk mengurangi ukuran memori dan file...
Ukuran memori awal DataFrame: 328.30 MB
Ukuran memori akhir DataFrame: 164.15 MB
Penghematan memori: 50.00%
Menyimpan data yang telah diproses ke: ../data/supervised_ready_data.csv (CSV)
Ukuran data yang disimpan (CSV): (558837, 77)

Contoh data yang telah diproses (5 baris pertama):


,year,condition,odometer,mmr,model,trim,seller,make_Audi,make_BMW,make_Cadillac,...,interior_off-white,interior_orange,interior_purple,interior_red,interior_silver,interior_tan,interior_white,interior_yellow,interior_—,sellingprice
0,1.250630,-1.940793,-0.967860,0.695349,0.572505,-0.066349,1.222651,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21500.0
1,1.250630,-1.940793,-1.103567,0.726342,0.572505,-0.066349,1.222651,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21500.0
2,0.998541,1.072405,-1.254557,1.873079,-2.508913,0.279639,0.356445,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30000.0
3,1.250630,0.771085,-1.012003,1.418517,0.572505,0.279639,-0.633504,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27750.0
4,0.998541,0.921745,-1.230022,5.395938,0.572505,0.279639,0.356445,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67000.0


# Simpan Preprocessor dan Label Encoders

In [ ]:
preprocessing_objects = {
    'preprocessor': preprocessor,
    'label_encoders': label_encoders,
    'processed_feature_names': processed_feature_names
}

print(f"Menyimpan objek preprocessor dan label encoders ke: {preprocessor_output_path}")
joblib.dump(preprocessing_objects, preprocessor_output_path)
print("Preprocessor dan Label Encoders berhasil disimpan.")

Menyimpan objek preprocessor dan label encoders ke: ../models/preprocessor_supervised.pkl
Preprocessor dan Label Encoders berhasil disimpan.
